In [19]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [20]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1.eval'
p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
p2t_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
p2t_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample.eval'
# context model
context_model = 'deberta-v3-small.flag-context4-2-d'
context_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'

In [41]:
df = pd.read_feather('../working/train.fea')

In [42]:
df.columns

Index(['cell_id', 'cell_type', 'source', 'id', 'n_words', 'cid', 'ancestor_id',
       'parent_id', 'n_cell', 'n_code_cell', 'n_markdown_cell',
       'markdown_frac', 'rank', 'code_rank', 'markdown_rank', 'rel_rank',
       'global_rank', 'local_rank', 'pct_rank', 'fold', 'worker'],
      dtype='object')

In [43]:
d = df[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank']]
d

,id,cell_id,rank,cell_type,rel_rank
0,00001756c60be8,1862f0a6,0,code,0.0323
1,00001756c60be8,448eb224,1,markdown,0.0484
2,00001756c60be8,2a9e43d6,2,code,0.0645
3,00001756c60be8,7e2f170a,3,markdown,0.0806
4,00001756c60be8,038b763d,4,code,0.0968
...,...,...,...,...,...
6370641,fffe1d764579d5,5369934b,67,markdown,0.9597
6370642,fffe1d764579d5,ac9db030,68,code,0.9677
6370643,fffe1d764579d5,a8ffc8b4,69,markdown,0.9758
6370644,fffe1d764579d5,70455170,70,code,0.9839


In [6]:
xp = gezi.load(f'{root}/{p2t_model}/valid.pkl')
dxp = pd.DataFrame(xp)

In [29]:
d = pd.DataFrame(xp)

In [21]:
# dxp

In [8]:
calc_metric(xp)

0.8892461693340118

In [9]:
calc_metric(xp, 'cls_pred')

0.8938159040179133

In [10]:
scores = calc_metrics(xp, 'cls_pred')
# scores

calc_metrics:   0%|          | 0/27590 [00:00<?, ?it/s]

In [11]:
dxp = to_df(xp, groupby=False, key='cls_pred')

In [12]:
# dxp

In [13]:
dxp = dxp[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank', 'pred']]

In [14]:
scores = scores.sort_values('score')

In [15]:
# scores

In [16]:
dxp.loc[dxp["cell_type"] == 'code',
        'pred'] = dxp.loc[dxp["cell_type"] == 'code', 'rel_rank']

In [50]:
x = pd.DataFrame(xp)

In [51]:
x[x.cid=='2d93ad6380413d\t252c288a']

,id,cell_id,cid,pred,max_prob,max_sim,cls_pred,match_rank,match_code,n_code_cell,n_cell,probs,sims
75023,2d93ad6380413d,252c288a,2d93ad6380413d\t252c288a,0.7353,0.5214,0.5529,0.6895,6,1,16,31,"[0.005887297, 0.0040418953, 0.004627625, 3.815212e-05, 0.0024742081, 0.0019751708, 0.029857986, 2.0738082e-05, 0.00032037188, 0.0022076017, 7.560369e-05, 0.41363215, 0.5213604, 0.009842683, 0.003538817, 5.7816742e-05, 4.081167e-05]","[0.46568388, 0.45836452, 0.46099833, 0.36761516, 0.4488127, 0.44442853, 0.49728355, 0.3557509, 0.40902847, 0.44659373, 0.38092571, 0.54844004, 0.55294484, 0.47568604, 0.45577762, 0.37570548, 0.36892664]"


In [52]:
probs = x[x.cid=='2d93ad6380413d\t252c288a'].probs.values[0]

In [53]:
idxes = (-probs).argsort()

In [54]:
idxes[:5]

array([12, 11,  6, 13,  0])

In [17]:
def vis(idx):
  print(scores.score.values[idx])
  id = scores.id.values[idx]
  d2 = dxp[dxp.id==id]
  gezi.plot.display_side_by_side((d[d.id==id], 'gt'), (d2.sort_values('pred'), 'pred'))
  d_ = df[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank', 'source']]
  d_ = d_[d_.id==id]
  display(d_)

In [49]:
scores = scores.sample(frac=1.)
vis(0)

0.5827956989247312


,id,cell_id,rank,cell_type,rel_rank
1139838,2d93ad6380413d,e8c82bed,0,code,0.0588
1139839,2d93ad6380413d,252c288a,1,markdown,0.0882
1139840,2d93ad6380413d,a9f17e55,2,code,0.1176
1139841,2d93ad6380413d,e099123d,3,code,0.1765
1139842,2d93ad6380413d,db9f76e5,4,markdown,0.2059
1139843,2d93ad6380413d,0919e9b7,5,code,0.2353
1139844,2d93ad6380413d,66690e87,6,code,0.2941
1139845,2d93ad6380413d,06123d78,7,markdown,0.3235
1139846,2d93ad6380413d,55a3b344,8,code,0.3529
1139847,2d93ad6380413d,60dba5a3,9,markdown,0.3824


,id,cell_id,rank,cell_type,rel_rank,source
1139838,2d93ad6380413d,e8c82bed,0,code,0.0588,pip install pyspark
1139839,2d93ad6380413d,252c288a,1,markdown,0.0882,# Importing patient data\n\nData is imported into spark dataframe
1139840,2d93ad6380413d,a9f17e55,2,code,0.1176,"from pyspark.sql import SparkSession\nfrom pyspark.sql.functions import col\nimport pandas as pd\n\npd.set_option('display.max_rows', 500)\npd.set_option('display.max_columns', 500)\npd.set_option('display.width', 150)\n\n\nspark = SparkSession \\n.builder \\n.appName(""Test data prediction"") \\n.getOrCreate()\n\nspark.conf.set(""spark.sql.execution.arrow.enabled"", ""false"")\nrawData = spark.read \\n.format('csv') \\n.option('header', 'true') \\n.load('../input/osic-pulmonary-fibrosis-progressi..."
1139841,2d93ad6380413d,e099123d,3,code,0.1765,"from pyspark.ml.stat import Summarizer\nfrom pyspark.ml.feature import VectorAssembler\n\nstat = VectorAssembler(\n inputCols= ['Percent', 'Age', 'FVC', 'Weeks'],\n outputCol= 'feature',\n handleInvalid=""keep""\n ).transform(rawData)\nsummarizer = Summarizer.metrics(""mean"", ""min"", ""max"", ""variance"")\nstat.select(summarizer.summary(stat.feature)).toPandas()"
1139842,2d93ad6380413d,db9f76e5,4,markdown,0.2059,# Correlation of Patient data
1139843,2d93ad6380413d,0919e9b7,5,code,0.2353,"from pyspark.ml.stat import Correlation\nimport matplotlib.pyplot as plt\nfrom pyspark.ml.feature import Normalizer\nimport seaborn as sns\nimport pandas as pd\n\nnormalizedFeature = Normalizer(inputCol='feature', outputCol='normalized feature').transform(stat)\npearsonCorr = Correlation.corr(normalizedFeature, 'normalized feature').collect()[0][0].toArray()\ndf = pd.DataFrame(pearsonCorr, columns= ['Percent', 'Age', 'FVC', 'Weeks'] )\nsns.heatmap(df, xticklabels= ['Percent', 'Age', 'FVC', '..."
1139844,2d93ad6380413d,66690e87,6,code,0.2941,#Function to get dicom data\nfrom pydicom.filebase import DicomBytesIO\nfrom pydicom.dataset import Dataset\n\ndef getDicomData(binry):\n dicom_bytes = DicomBytesIO(binry)\n return dicom_bytes\n
1139845,2d93ad6380413d,06123d78,7,markdown,0.3235,# Image feature extraction\n\nI am using opencv as image processing tool.
1139846,2d93ad6380413d,55a3b344,8,code,0.3529,"#Function to get and display plotted contours\nimport numpy as np\nimport cv2\nimport matplotlib.pyplot as plt\nfrom pydicom import dcmread\n\n\ndef getLungContours(data, display = False):\n \n try:\n ds = dcmread(data, force = True)\n except Exception:\n return ""Unknown"", 0, list(np.array([[[0, 0]]]))\n \n try:\n pixel_data = ds.pixel_array\n except RuntimeError:\n pixel_data = np.ones((512, 512))\n \n img_2d = pixel_data.astype(floa..."
1139847,2d93ad6380413d,60dba5a3,9,markdown,0.3824,**Extracted Contour information from CT scan image**
